**IMPORTING LIBRARIES:**

In [257]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

**Reading Company_X Reports:**

In [258]:
x_order_report=pd.read_excel("Company X - Order Report.xlsx")
x_order_report.head()

,ExternOrderNo,SKU,Order Qty,Payment Mode,Item Price(Per Qty.)
0,2001827036,8904223818706,1,COD,233
1,2001827036,8904223819093,1,COD,233
2,2001827036,8904223819109,1,COD,233
3,2001827036,8904223818430,1,COD,470
4,2001827036,8904223819277,1,COD,233


In [259]:
x_pincode_zone=pd.read_excel("Company X - Pincode Zones.xlsx")
x_pincode_zone.head()


,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d


In [260]:
x_sku=pd.read_excel("Company X - SKU Master.xlsx")
x_sku.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


**Reading Courier Company Reports:**

In [261]:
courier_invoice=pd.read_excel("Courier Company - Invoice.xlsx") #Reading Courier company reports
courier_invoice.head()

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,140.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,101.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4


In [262]:
courier_rates=pd.read_excel("Courier Company - Rates.xlsx")
courier_rates.head()

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.25,29.5,23.6,13.6,23.6
1,B,0.50,33.0,28.3,20.5,28.3
2,C,0.75,40.1,38.9,31.9,38.9
3,D,1.25,45.4,44.8,41.3,44.8
4,E,1.50,56.6,55.5,50.7,55.5


**Sample output:**

In [263]:
sample_result=pd.read_excel("Expected_Result.xlsx",sheet_name="Calculations") #reading calculation sheet in Expected_Result.xlsx
pd.set_option('display.width', None) 
sample_result

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.30,2.5,1.30,2.5,D,D,90.2,90.2,0.0
1,2001806210,1091117221940,0.22,0.5,2.92,3.0,B,B,33.0,174.5,-141.5


In [264]:
df=pd.merge(x_order_report,x_sku,on="SKU",how="inner")
print(df)

     ExternOrderNo            SKU  Order Qty Payment Mode  \
0       2001827036  8904223818706          1          COD   
1       2001821995  8904223818706          1      Prepaid   
2       2001819252  8904223818706          1      Prepaid   
3       2001816996  8904223818706          1      Prepaid   
4       2001814580  8904223818706          1          COD   
..             ...            ...        ...          ...   
396     2001806616  8904223819123          1          COD   
397     2001806567  8904223815804          1          COD   
398     2001806567  8904223818577          1          COD   
399     2001806408  8904223819437          2      Prepaid   
400     2001806232  8904223818645          6          COD   

     Item Price(Per Qty.)  Weight (g)  
0                     233         127  
1                     233         127  
2                     233         127  
3                     233         127  
4                     233         127  
..                    ...  

**Changing  Weights into KG**

In [265]:
df["Weight (kg)"]=df["Weight (g)"]/1000

In [266]:
df["Total Price as per X (KG)"]=df["Order Qty"]*df["Item Price(Per Qty.)"] #Calculated the total price

In [267]:
df["Total weight as per X (KG)"]=df["Order Qty"]*df["Weight (kg)"]  #Calculated the Total Weight

In [268]:
df.rename(columns={"ExternOrderNo":"Order ID"},inplace=True)    #renaming column to merge.

In [269]:
x_pincode_zone.rename(columns={"Zone":"Delivery Zone as per X"}, inplace=True)  #renaming column to merge.

**Merging datasets:**

In [270]:
df2=pd.merge(df,courier_invoice,on="Order ID",how="inner")

In [271]:
df3=pd.merge(df2,x_pincode_zone, on="Customer Pincode",how="inner")

**Create a function to assaign Permissible Slots**

In [272]:
# weight slab assaign
def weight_slab_assaign(zone):
    if zone == "a":
        return 0.25
    elif zone == "b":
        return 0.5
    elif zone == "c":
        return 0.75
    elif zone == "d":
        return 1.25
    elif zone == "e":
        return 1.5

In [273]:
df3["All_Weight slab"]=df3["Delivery Zone as per X"].apply(weight_slab_assaign)

**Calculating Sufficient Slabs:**

In [274]:
df3["Weight slab as per X (KG)"]=((df3["Total weight as per X (KG)"]//df3["All_Weight slab"])+1)*df3["All_Weight slab"]


In [275]:
df3["C_All_Weight slab"]=df3["Zone"].apply(weight_slab_assaign)

**Calculating Sufficient Slabs:**

In [276]:
df3["Weight slab charged by Courier Company (KG)"]=((df3["Charged Weight"]//df3["C_All_Weight slab"])+1)*df3["C_All_Weight slab"]

**Calculating No of Additional Slabs:**

In [277]:
df3["Additional_Slabs_asper_X"]=(df3["Weight slab as per X (KG)"]/df3["All_Weight slab"])-1

In [278]:
df3["Additional_Slabs_asper_X"].value_counts()

0.0    559
1.0      8
Name: Additional_Slabs_asper_X, dtype: int64

**Calculating Charge as per company:**

In [279]:
def X_charge(row):
    if row["Delivery Zone as per X"]=="a":
        if row["Type of Shipment"]=="Forward charges":
            charge = 29.5 + (row["Additional_Slabs_asper_X"] * 23.6)
        elif row["Type of Shipment"]=="Forward and RTO charges":
            charge = (29.5 + (row["Additional_Slabs_asper_X"] * 23.6)) + (13.6 + row["Additional_Slabs_asper_X"] * 23.6)
    if row["Delivery Zone as per X"]=="b":
        if row["Type of Shipment"]=="Forward charges":
            charge = 33 + (row["Additional_Slabs_asper_X"] * 28.3)
        elif row["Type of Shipment"]=="Forward and RTO charges":
            charge = (33 + (row["Additional_Slabs_asper_X"] * 28.3)) + (20.5 + row["Additional_Slabs_asper_X"] * 28.3)
    if row["Delivery Zone as per X"]=="c":
        if row["Type of Shipment"]=="Forward charges":
            charge = 40.1 + (row["Additional_Slabs_asper_X"] * 38.9)
        elif row["Type of Shipment"]=="Forward and RTO charges":
            charge = (40.1 + (row["Additional_Slabs_asper_X"] * 38.9)) + (31.9 + row["Additional_Slabs_asper_X"] * 38.9)
    if row["Delivery Zone as per X"]=="d":
        if row["Type of Shipment"]=="Forward charges":
            charge = 45.4 + (row["Additional_Slabs_asper_X"] * 44.8)
        elif row["Type of Shipment"]=="Forward and RTO charges":
            charge = (45.4 + (row["Additional_Slabs_asper_X"] * 44.8)) + (41.3 + row["Additional_Slabs_asper_X"] * 44.8)
    if row["Delivery Zone as per X"]=="e":
        if row["Type of Shipment"]=="Forward charges":
            charge = 56.6 + (row["Additional_Slabs_asper_X"] * 55.5)
        elif row["Type of Shipment"]=="Forward and RTO charges":
            charge = (56.6 + (row["Additional_Slabs_asper_X"] * 55.5)) + (50.7 + row["Additional_Slabs_asper_X"] * 55.5)
    return(charge)

In [280]:
df3["Charge"]=df3.apply(X_charge,axis=1)

In [281]:
df3["Charge"].value_counts()

33.0     372
45.4     108
56.6      29
86.7      28
53.5      17
61.3       8
107.3      5
Name: Charge, dtype: int64

**Calculating Total Charge as per the Company**

In [282]:
def tot_charge(row):
    if row["Payment Mode"]=="Prepaid":
        return(row["Charge"])
    if row["Payment Mode"]=="COD":
        if row["Charge"]<=300:
            return(row["Charge"] + 15)
        else:
            return(row["Charge"] * 1.05)

In [283]:
df3["Expected Charge as per X (Rs.)"]=df3.apply(tot_charge, axis=1)

In [284]:
df3["Expected Charge as per X (Rs.)"].value_counts()

33.0     187
48.0     185
45.4      70
60.4      38
71.6      26
86.7      21
68.5      11
101.7      7
53.5       6
122.3      5
61.3       4
76.3       4
56.6       3
Name: Expected Charge as per X (Rs.), dtype: int64

**Renaming the column names to get desired output:**

In [285]:
df3.rename(columns={"AWB Code":"AWB Number","Charged Weight":"Total weight as per Courier Company (KG)","Zone":"Delivery Zone charged by Courier Company","Billing Amount (Rs.)":"Charges Billed by Courier Company (Rs.)"},inplace=True)

**Calculating Difference Between Expected Charges and Billed Charges (Rs.)**

In [286]:
df3["Difference Between Expected Charges and Billed Charges (Rs.)"]=df3["Expected Charge as per X (Rs.)"]-df3["Charges Billed by Courier Company (Rs.)"]

In [287]:
columns=["AWB Number", "Total weight as per X (KG)", "Weight slab as per X (KG)","Total weight as per Courier Company (KG)","Weight slab charged by Courier Company (KG)", "Delivery Zone as per X","Delivery Zone charged by Courier Company","Expected Charge as per X (Rs.)","Charges Billed by Courier Company (Rs.)","Difference Between Expected Charges and Billed Charges (Rs.)"]

**Getting desired dataframe as output:**

In [253]:
final_df=df3[columns]

In [254]:
final_df

,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,1091122418320,0.127,1.50,1.60,2.0,e,b,71.6,117.9,-46.3
1,1091122418320,0.150,1.50,1.60,2.0,e,b,71.6,117.9,-46.3
2,1091122418320,0.100,1.50,1.60,2.0,e,b,71.6,117.9,-46.3
3,1091122418320,0.165,1.50,1.60,2.0,e,b,71.6,117.9,-46.3
4,1091122418320,0.350,1.50,1.60,2.0,e,b,71.6,117.9,-46.3
...,...,...,...,...,...,...,...,...,...,...
562,1091117222146,0.115,1.25,1.27,2.5,d,d,45.4,135.0,-89.6
563,1091117222146,0.130,1.25,1.27,2.5,d,d,45.4,135.0,-89.6
564,1091117222135,0.115,0.50,0.78,1.0,b,b,33.0,61.3,-28.3
565,1091117222135,0.130,0.50,0.78,1.0,b,b,33.0,61.3,-28.3


**Save to the Excel Format:**

In [256]:
final_df.to_excel("result.xlsx")